# 📊 Automating the Collection of EXPLAIN Plans and Runtime Metrics in Db2 LUW
This notebook automates the process of running a SQL query in Db2 and capturing its execution details using EXPLAIN tables and activity event monitors. The workflow includes:

1. **Setup of Activity Event Monitor:** Configures an event monitor in Db2 to collect runtime metrics such as execution time and resource usage during query execution.
2. **Creation of EXPLAIN Tables:** Sets up EXPLAIN tables in Db2 to store detailed execution plans for SQL queries.
3. **Query Execution:** Runs a SQL query to gather execution metrics and analyze its performance.
4. **EXPLAIN Plan Generation:** Captures the execution plan of the query to understand how Db2 will process the operation.
5. **Exporting Data:** Exports the collected EXPLAIN and activity monitor data as CSV files for external analysis or reporting.

For detailed setup instructions and guidance on running this notebook, refer to the [README.md](./README.md) file in the same directory.

This notebook provides a fully automated approach for analyzing query performance and gathering runtime metrics, helping database administrators and developers optimize their queries effectively.


In [1]:
import os
from dotenv import dotenv_values
import json
import shutil
import pandas as pd

In [2]:
# Parameters (These values will be overwritten when called by papermill)
queryid = None
sql_statement = None

In [3]:
# Parameters
queryid = "query1"
sql_statement = "SELECT SS_WHOLESALE_COST , SR_NET_LOSS FROM STORE_SALES INNER JOIN STORE_RETURNS ON SR_TICKET_NUMBER = SS_TICKET_NUMBER AND SR_ITEM_SK = SS_ITEM_SK WHERE SS_STORE_SK = 2 AND SR_RETURNED_DATE_SK = 2451680"


Loading Db2 Magic Commands Notebook Extension

In [4]:
# Enable Db2 Magic Commands Extensions for Jupyter Notebook
if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')
%run db2.ipynb

<>:1708: SyntaxWarning: invalid escape sequence '\s'
<>:2305: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipykernel_432137/2299624180.py:1708: SyntaxWarning: invalid escape sequence '\s'
  firstCommand = "(?:^\s*)([a-zA-Z]+)(?:\s+.*|$)"
/tmp/ipykernel_432137/2299624180.py:2305: SyntaxWarning: invalid escape sequence '\?'
  pattern = "\?\*[0-9]+"


Db2 Extensions Loaded. Version: 2024-09-16


In [5]:
import os

# Define the top-level output directory
top_level_dir = os.path.join(os.getcwd(), "output")

# Create the top-level directory only if it doesn't exist
os.makedirs(top_level_dir, exist_ok=True)

Connect to Db2

In [6]:
db2creds = dotenv_values('.env')
%sql CONNECT CREDENTIALS db2creds

Connection successful. tpcds @ localhost 


In [7]:
# Activate event monitor
%sql SET EVENT MONITOR ACTEVMON STATE 1;

Command completed.


In [8]:
# Set client info
%sql CALL WLM_SET_CLIENT_INFO(NULL,NULL,NULL,:queryid,NULL);

[None, None, None, 'query1', None]

In [9]:
%sql {sql_statement}

,SS_WHOLESALE_COST,SR_NET_LOSS
0,68.54,473.29
1,47.99,1448.18
2,67.78,557.29
3,3.62,140.00
4,29.61,NaN
5,26.28,121.02
6,45.44,55.13
7,88.22,522.09
8,74.79,57.54
9,93.95,5798.67


In [10]:
# Deactivate the event monitor to ensure its data is written to the activity tables
%sql SET EVENT MONITOR ACTEVMON STATE 0

Command completed.


In [11]:
result = %sql SELECT a.APPL_ID, a.UOW_ID, a.ACTIVITY_ID \
    FROM ACTIVITY_ACTEVMON a \
    WHERE a.ACTIVITY_TYPE = 'READ_DML' AND a.TPMON_ACC_STR = :queryid

In [12]:
appl_id = result['APPL_ID'].iloc[0]
uow_id = result.at[0, 'UOW_ID'].item()
activity_id = result.at[0, 'ACTIVITY_ID'].item()
event_monitor = 'ACTEVMON'
schema = 'DB2INST1'

In [13]:

print(result)

                        APPL_ID  UOW_ID  ACTIVITY_ID
0  127.0.0.1.52522.250225035408       5            1


In [14]:
print('appl_id: ', appl_id)
print('uow_id: ', uow_id)
print('activity_id: ', activity_id)

appl_id:  127.0.0.1.52522.250225035408
uow_id:  5
activity_id:  1


In [15]:
%%capture explain_output
sql = f'''"CALL EXPLAIN_FROM_ACTIVITY('{appl_id}', '{uow_id}', '{activity_id}', '{event_monitor}', '{schema}', null, null, null, null, null)"'''
_ = ! db2 "connect to TPCDS" 

explain = %system db2 {sql}

In [16]:
# Initialize variables
explain_time = None
source_name = None
source_schema = None

# Iterate through the list
for i in range(len(explain)):
    if "EXPLAIN_TIME" in explain[i]:
        explain_time = explain[i + 1].split(":")[-1].strip()
    elif "SOURCE_NAME" in explain[i]:
        source_name = explain[i + 1].split(":")[-1].strip()
    elif "SOURCE_SCHEMA" in explain[i]:
        source_schema = explain[i + 1].split(":")[-1].strip()

# Print extracted values
print("EXPLAIN_TIME:", explain_time)
print("SOURCE_NAME:", source_name)
print("SOURCE_SCHEMA:", source_schema)

EXPLAIN_TIME: 2025-02-24-19.54.16.324073
SOURCE_NAME: SYSSH200
SOURCE_SCHEMA: NULLID


In [17]:
explain_time

'2025-02-24-19.54.16.324073'

In [18]:
from dotenv import dotenv_values

# Load environment variables from the .env file
db2creds = dotenv_values('.env')

# Extract the database name
database_name = db2creds.get("database")  # Use .get() to avoid KeyError if the key is missing

# Print the extracted database name
print("Database Name:", database_name)

Database Name: tpcds


# Generate Explain and Export the Explain output

In [19]:
# Define the subdirectory for a specific explain_time output
outputdir = os.path.join(top_level_dir, f"{explain_time}")
os.makedirs(outputdir, exist_ok=True)  # Create the subdirectory if it doesn't exist

# If the directory exists, delete its contents
for filename in os.listdir(outputdir):
    file_path = os.path.join(outputdir, filename)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)  # Remove files and symlinks
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)  # Remove subdirectories

# Define output file
explain_output = "explain.out"

db2exfmt_cmd = f'''db2exfmt -d "{database_name}" -w "{explain_time}" -n "{source_name}" -s "{source_schema}" -# 0 -o "{outputdir}/{explain_output}"'''
print(db2exfmt_cmd)

# Uncomment the following line to execute the command (if running in a shell environment)
os.system(db2exfmt_cmd)

# Load the dictionary from the JSON file
with open("export_sql.json", "r") as json_file:
    export_sql_statements = json.load(json_file)

print("SQL statements have been loaded from sql_statements.json")

# Loop through each table in the dictionary
for table_name, query in export_sql_statements.items():
    print(f"Processing table: {table_name}")

    # Execute the dynamically generated SQL using %sql magic
    df_result = %sql {query}

    # Convert result to Pandas DataFrame
    # df_result = df_result.DataFrame()

    # Save to CSV without including the index in the "explain" directory
    output_file_path = os.path.join(outputdir, f"{table_name}.csv")
    df_result.to_csv(output_file_path, index=False)

    print(f"Saved {table_name} data to {output_file_path}")

db2exfmt -d "tpcds" -w "2025-02-24-19.54.16.324073" -n "SYSSH200" -s "NULLID" -# 0 -o "/home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/explain.out"


DB2 Universal Database Version 11.5, 5622-044 (c) Copyright IBM Corp. 1991, 2019
Licensed Material - Program Property of IBM
IBM DATABASE 2 Explain Table Format Tool

Connect to Database Successful.


Output is in /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/explain.out.
Executing Connect Reset -- Connect Reset was Successful.


Connecting to the Database.
SQL statements have been loaded from sql_statements.json
Processing table: ACTIVITYSTMT_ACTEVMON
Saved ACTIVITYSTMT_ACTEVMON data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/ACTIVITYSTMT_ACTEVMON.csv
Processing table: ACTIVITY_ACTEVMON


Saved ACTIVITY_ACTEVMON data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/ACTIVITY_ACTEVMON.csv
Processing table: EXPLAIN_ACTUALS
Saved EXPLAIN_ACTUALS data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/EXPLAIN_ACTUALS.csv
Processing table: EXPLAIN_INSTANCE
Saved EXPLAIN_INSTANCE data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/EXPLAIN_INSTANCE.csv
Processing table: EXPLAIN_OBJECT
Saved EXPLAIN_OBJECT data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/EXPLAIN_OBJECT.csv
Processing table: EXPLAIN_OPERATOR
Saved EXPLAIN_OPERATOR data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/EXPLAIN_OPERATOR.csv
Processing table: EXPLAIN_PREDICATE
Saved EXPLAIN_PREDICATE data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/EXPLAIN_PREDICATE.csv
Processing table: EXPLAIN_STATEMENT
S

Saved EXPLAIN_STREAM data to /home/db2inst1/db2-labs/explain/batch-queries/output/2025-02-24-19.54.16.324073/EXPLAIN_STREAM.csv


In [20]:
%sql CONNECT RESET

Connection closed.
